In [1]:
import cirq
import openfermion as of
import sys
sys.path.append('../utils/')
from basis_utils import InnProd
from qiskit import transpile, QuantumCircuit
from qiskit.qasm2 import dumps
from cirq.contrib.qasm_import import circuit_from_qasm
import scipy
from openfermion import QubitOperator
from basis_utils import S_plus, S_minus, MatRepLib
from basis_utils import read_spinach_info, build_list_ISTs, NormalizeBasis, MatRepLib,Sz,Sx,Sy#,HB_norm
from direct_sim_utils import HamMatRep, BuildUCirc, Build2D_circ, Meas_TotMag2

import numpy as np
import scipy.io as spio
from scipy.linalg import expm
import copy

In [2]:
####These are the Spinach-generated inputs needed for the analytical generation of relaxation matrix...
text="""1      (0,0)   (0,0)   
  2      (0,0)   (1,1)   
  3      (0,0)   (1,0)   
  4      (0,0)   (1,-1)  
  5      (1,1)   (0,0)   
  6      (1,1)   (1,1)   
  7      (1,1)   (1,0)   
  8      (1,1)   (1,-1)  
  9      (1,0)   (0,0)   
  10     (1,0)   (1,1)   
  11     (1,0)   (1,0)   
  12     (1,0)   (1,-1)  
  13     (1,-1)  (0,0)   
  14     (1,-1)  (1,1)   
  15     (1,-1)  (1,0)   
  16     (1,-1)  (1,-1)  
"""
gammaF = 251814800
coord1 = np.array([-0.0551,-1.2087,-1.6523])*1e-10
coord2 = np.array([-0.8604 ,-2.3200 ,-0.0624])*1e-10

coords = np.array([coord1,coord2])

w1 = -376417768.6316 
w2 = -376411775.1523 
freqs = np.array([w1,w2])
tc = 0.5255e-9

data = read_spinach_info(text)

basis = build_list_ISTs(data)

Normbasis = NormalizeBasis(basis,n_qubits=2,checkOrth=True)



loadMat = spio.loadmat('../data/DFG.mat',squeeze_me=True)

AuxMats = spio.loadmat('../data/DFG_NOESYmatrices.mat',squeeze_me=True)


Ham = loadMat['p']['H'].item()

In [3]:
offset = -46681
B0 = 9.3933
zeeman_scalar_1 = -113.8796
zeeman_scalar_2 = -129.8002
Jcoup = 2*np.pi*238.0633


w0 = -gammaF*B0

o1 = 2*np.pi*offset
w1 = o1+w0*zeeman_scalar_1/1e6
w2 = o1+w0*zeeman_scalar_2/1e6

#
Hamiltonian = w1*Sz(0)+w2*Sz(1)+Jcoup*(Sx(0)*Sx(1)+Sy(0)*Sy(1)+Sz(0)*Sz(1))

#Checking that the matrix representation of the Hamiltonian matches that of Spinach...

Mat_H = HamMatRep(Hamiltonian,Normbasis)

print("Difference between analytically generated matrx Rep of Hamiltonian and that of Spinach:", np.linalg.norm(Mat_H-Ham))

Difference between analytically generated matrx Rep of Hamiltonian and that of Spinach: 6.28069997787362e-07


In [4]:
#Quick test for computation of expectation values....

###Ramping up relaxation rate to detect changes of total magnetization for the timescales of propagation....

t2_mod = 1000
tmix = 0.5

T2_Chan_1 = 2*t2_mod*MatRepLib(Normbasis,Sz(0),Sz(0))

Mod_Ham = Ham+1j*T2_Chan_1


JumpOp = np.sqrt(2*t2_mod)*Sz(0)

#The Hamiltonian to simulate, given the Hermitian character of the jump operator is 

Heff = Hamiltonian+of.QubitOperator('X2')*JumpOp

spHeff = of.get_sparse_operator(Heff,n_qubits=3)

deltaT = 1e-4 #we need to use small enough delta values, this is of the order of dt1,dt2

dilT1 = np.sqrt(deltaT)
dilT2 = np.sqrt(deltaT)
diltTmix = np.sqrt(tmix)

DilHam_1 = expm(-1j*spHeff.toarray()*dilT1)
DilHam_2 = expm(-1j*spHeff.toarray()*dilT2)


PropPauliBasis = expm(-1j*Mod_Ham*deltaT)

Lx_qub = Sx(0)+Sx(1)
Ly_qub = Sy(0)+Sy(1)


sp_Lx_qub = of.get_sparse_operator(Lx_qub,n_qubits=3)
sp_Ly_qub = of.get_sparse_operator(Ly_qub,n_qubits=3)
pulse_90x_direct = expm(-1j*(np.pi/2.0)*sp_Lx_qub.toarray())
pulse_90y_direct =  expm(-1j*(np.pi/2)*sp_Ly_qub.toarray())
pulse_90mx_direct = expm(1j*(np.pi/2.0)*sp_Lx_qub.toarray())
pulse_90my_direct = expm(1j*(np.pi/2.0)*sp_Ly_qub.toarray())
pulse_mix_direct = expm(-1j*spHeff.toarray()*diltTmix)

#unitary that includes the first pulse as part of state preparation, notice that we need to sample over different initial states, determined by a thermal distribution...
#Unitary = Unitary_Ham@pulse_90x_direct

#TODO: 1) construct the "time-snapshots" for propagation at different times of the magnetization
#2) ensemble average these estimations according to the initial distribution
#3) compare with a reference value...



In [ ]:

t1_steps = 0
t2_steps = 0
tmix_steps = 1
allqubits = cirq.LineQubit.range(3)
anc_register = allqubits[2:3]
CircInit = BuildUCirc(np.eye(2**(3)),allqubits)

CircPulse_90x = BuildUCirc(pulse_90x_direct,allqubits)
CircPulse_90y = BuildUCirc(pulse_90y_direct,allqubits)
CircPulse_90mx = BuildUCirc(pulse_90mx_direct,allqubits)
CircPulse_90my = BuildUCirc(pulse_90my_direct,allqubits)

#For debugging purposes....


TestFID_1CircX, TestFID_1CircY = Build2D_circ(t1_steps,t2_steps,tmix_steps,CircInit,DilHam_1,DilHam_2,CircPulse_90x,CircPulse_90x,CircPulse_90y,allqubits,anc_register)

In [ ]:
###Estimate the total magntization along X...
Meas_TotMag2(TestFID_1CircY,allqubits,2,100)

Expect value for spin 0 is -0.16
Expect value for spin 1 is -0.08


np.float64(-0.24)